In [4]:
#sudo apt install git
#sudo apt install python3 python3-pip
#$ python3 -m pip install ipykernel
#pip list

In [13]:
import dash
from dash import html 
from dash import dcc
from dash.dependencies import Input, Output

from datetime import datetime, timedelta, date

from meteostat import Point, Daily


import plotly.express as px
import dash_leaflet as dl

from jupyter_dash import JupyterDash

In [18]:
#print the versions of the libraries
print(f'DASH version: {dash.__version__}')
print(f'HTML version: {html.__version__}')

import meteostat
print(f'Meteostat version: {meteostat.__version__}')

import plotly
print(f'Plotly version: {plotly.__version__}')
print(f'Dash Leaflet version: {dl.__version__}')

import jupyter_dash
print(f'Jupyter DASH version: {jupyter_dash.__version__}')

DASH version: 2.7.0
HTML version: 2.0.6
Meteostat version: 1.6.5
Plotly version: 5.11.0
Dash Leaflet version: 0.1.23
Jupyter DASH version: 0.4.2


## Open-Meteo

In [10]:
#pip install openmeteo-py
#https://pypi.org/project/openmeteo-py/

In [11]:
#pip uninstall open-meteo


In [5]:
from openmeteo_py import Hourly,Daily,Options,OWmanager

# Latitude, Longitude for Rabat,Morocco
latitude = 52.2
longitude = 20

hourly = Hourly()
daily = Daily()
options = Options(latitude,longitude)

mgr = OWmanager(options,
    hourly.all(),
    daily.all())


# Download data
meteo = mgr.get_data()

print(meteo)
print(type(meteo))


ModuleNotFoundError: No module named 'openmeteo_py.Daily'

In [17]:
def get_schema(dictionary):
    schema = {}
    for key, value in dictionary.items():
        schema[key] = type(value).__name__
    return schema

dictionary = {
    'name': 'John',
    'age': 30,
    'pets': ['dog', 'cat'],
    'married': True,
}

#print(get_schema(dictionary))
print(get_schema(meteo))

{'latitude': 'float', 'longitude': 'float', 'generationtime_ms': 'float', 'utc_offset_seconds': 'int', 'timezone': 'str', 'timezone_abbreviation': 'str', 'elevation': 'float', 'hourly_units': 'dict', 'hourly': 'dict', 'daily_units': 'dict', 'daily': 'dict'}


In [18]:
def get_schema(dictionary, parent_key=''):
    schema = {}
    for key, value in dictionary.items():
        full_key = f'{parent_key}.{key}' if parent_key else key
        if isinstance(value, dict):
            schema.update(get_schema(value, full_key))
        else:
            schema[full_key] = type(value).__name__
    return schema

# Usage
schema = get_schema(meteo)
for key, value in schema.items():
    print(f"{key}: {value}")


latitude: float
longitude: float
generationtime_ms: float
utc_offset_seconds: int
timezone: str
timezone_abbreviation: str
elevation: float
hourly_units.time: str
hourly_units.relativehumidity_2m: str
hourly_units.dewpoint_2m: str
hourly_units.apparent_temperature: str
hourly_units.pressure_msl: str
hourly_units.cloudcover: str
hourly_units.cloudcover_low: str
hourly_units.cloudcover_mid: str
hourly_units.cloudcover_high: str
hourly_units.windspeed_10m: str
hourly_units.windspeed_80m: str
hourly_units.windspeed_120m: str
hourly_units.windspeed_180m: str
hourly_units.winddirection_10m: str
hourly_units.winddirection_80m: str
hourly_units.winddirection_120m: str
hourly_units.winddirection_180m: str
hourly_units.windgusts_10m: str
hourly_units.shortwave_radiation: str
hourly_units.direct_radiation: str
hourly_units.diffuse_radiation: str
hourly_units.vapor_pressure_deficit: str
hourly_units.evapotranspiration: str
hourly_units.precipitation: str
hourly_units.weathercode: str
hourly_units.

In [19]:
import pandas as pd

# assuming that your dictionary is called `meteo`
daily_data = meteo["daily"]

# check if 'time' and 'apparent_temperature_max' are in daily_data
if 'time' in daily_data and 'apparent_temperature_max' in daily_data:
    df = pd.DataFrame({
        'time': daily_data['time'],
        'apparent_temperature_max': daily_data['apparent_temperature_max']
    })
else:
    print("'time' or 'apparent_temperature_max' not found in daily data")


In [21]:
import pandas as pd

# assuming that your dictionary is called `meteo`
daily_data = meteo["daily"]

# check if 'time', 'apparent_temperature_max' and 'apparent_temperature_min' are in daily_data
if 'time' in daily_data and 'apparent_temperature_max' in daily_data and 'apparent_temperature_min' in daily_data:
    df_meteo = pd.DataFrame({
        'time': daily_data['time'],
        'apparent_temperature_max': daily_data['apparent_temperature_max'],
        'apparent_temperature_min': daily_data['apparent_temperature_min']
    })
else:
    print("'time', 'apparent_temperature_max' or 'apparent_temperature_min' not found in daily data")


In [22]:
df_meteo

,time,apparent_temperature_max,apparent_temperature_min
0,2023-06-22,28.2,16.3
1,2023-06-23,29.6,14.6
2,2023-06-24,20.6,14.3
3,2023-06-25,25.4,12.6
4,2023-06-26,26.6,12.6
5,2023-06-27,18.7,12.0
6,2023-06-28,22.2,11.1


## Meteostat

In [ ]:
#pip install meteostat
#https://github.com/meteostat/meteostat-python

In [9]:
### The Stations Class: latitude, longitude
#https://dev.meteostat.net/python/stations.html

# Import Meteostat library
from meteostat import Stations

# Get nearby weather stations
stations = Stations()
stations = stations.nearby(36.92, 30.72) 
station = stations.fetch(1) #the closest station to the input location

# Print DataFrame
print(station)
print(type(station))

          name country region    wmo  icao  latitude  longitude  elevation  \
id                                                                           
17300  Antalya      TR    ANT  17300  LTAI      36.7    30.7333       50.0   

              timezone hourly_start hourly_end daily_start  daily_end  \
id                                                                      
17300  Europe/Istanbul   1951-01-01 2023-06-20  1951-01-01 2023-06-14   

      monthly_start monthly_end      distance  
id                                             
17300    1949-01-01  2022-01-01  24491.521703  
<class 'pandas.core.frame.DataFrame'>


In [16]:
nearest_alt = station['latitude'].values[0]
nearest_lon = station['longitude'].values[0]

In [14]:
### The Point Class: latitude, longitude, altitude
#https://dev.meteostat.net/python/point.html

from meteostat import Point, Daily
from datetime import datetime

import plotly.express as px


# Set time period
start = datetime(2022, 1, 1)
end = datetime(2022, 12, 31)

# Create Point (KOS island in Greece)
#location = Point(36.8925871 , 27.2877926, 70) #lat, long, altitude (optional)
location = Point(36.8925871 , 27.2877926)

# Get daily data
data = Daily(location, start, end)
data = data.fetch()

print(data)
print(type(data))

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2022-01-01  14.1  12.0  16.1   NaN   NaN  335.0  11.2  13.0  1018.7   NaN
2022-01-02  14.3  11.9  16.4   NaN   NaN    0.0  18.1  22.2  1021.5   NaN
2022-01-03  14.2  11.4  17.9   NaN   NaN  254.0   9.9  16.7  1019.5   NaN
2022-01-04  14.4  12.6  17.7   NaN   NaN  226.0  10.4  18.5  1015.5   NaN
2022-01-05  15.8  12.6  18.6   NaN   NaN  199.0  25.2  20.4  1014.0   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2022-12-27  13.6  10.7  18.3   0.0   NaN  220.0   3.8  16.7  1024.4   NaN
2022-12-28  13.6  11.2  16.8   0.0   NaN    7.0   3.8  13.0  1024.2   NaN
2022-12-29  13.5  10.7  16.4   0.0   NaN  356.0   7.1  13.0  1025.4   NaN
2022-12-30  12.6   9.8  16.8   1.3   NaN  173.0   2.2  16.7  1026.5   NaN
2022-12-31  13.0  10.4  16.3   4.3   NaN  315.0   3.5  14.8  1028.4   NaN

[365 rows x 10 columns]
<class 'panda

In [8]:
fig = px.line(data.reset_index(), x='time', y=['tmin','tmax'],
                title=f'Temperature Min/Max for the Given Location',
                labels={'time': 'Date', 'tmin': 'Min Temp', 'tmax': 'Max Temp'})

fig.data[0].name = 'Tmin'
fig.data[1].name = 'Tmax'


# Update the legend title
fig.update_layout(legend_title='Legend')

fig.update_yaxes(title_text='Temperature (°C)')

# Center the plot title
fig.update_layout(title={'x': 0.5, 'xanchor': 'center'})

fig.show()

In [10]:
# imports
import plotly
# Export as static image

# # Assuming your plot is in the 'fig' variable
#fig.write_image("yourfile.png") 


# Export as HTML

# # Assuming your plot is in the 'fig' variable
plotly.offline.plot(fig, filename='./images/Temperature-Trend.html')

'./images/Temperature-Trend.html'

In [6]:
fig = px.line(data.reset_index(), x='time', y=['wspd'],
                title=f'Windspeed for the Given Location',
                labels={'time': 'Date', 'wspd': 'Wind Speed'})

fig.data[0].name = 'Wind'

# Update the legend title
fig.update_layout(legend_title='Legend')

fig.update_yaxes(title_text='Wind Speed (km/h)')

# Center the plot title
fig.update_layout(title={'x': 0.5, 'xanchor': 'center'})

fig.show()

In [5]:
fig = px.line(data.reset_index(), x='time', y=['prcp'],
              title=f'Precipitation for the Given Location',
              labels={'time': 'Date', 'prcp': 'Rain'})

fig.data[0].name = 'Rain'

fig.update_yaxes(title_text='Rainfall (mm)')

# Update the legend title
fig.update_layout(legend_title='Legend')

# Center the plot title
fig.update_layout(title={'x': 0.5, 'xanchor': 'center'})

fig.show()


In [16]:
# from datetime import date, timedelta

# today = date.today()
# five_days_ago = date.today() - timedelta(days=5)
# print(five_days_ago)

In [60]:
lat = 35.0
lon = 25.0
start = datetime(2021, 1, 1)
end = datetime(2021, 12, 31)


location = Point(lat, lon, 70)

data = Daily(location, start, end)
data = data.fetch()

data = data.reset_index()

print("\nData columns:\n", data.columns)


Data columns:
 Index(['time', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
       'pres', 'tsun'],
      dtype='object')


In [61]:
# Rename the columns
print(data.rename(columns={'tmin': 'Tmin',
                           'tmax': 'Tmax'}))	

          time  tavg  Tmin  Tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
0   2021-01-01  13.6   9.5  18.4   NaN   NaN  108.0   6.9   NaN  1016.6   NaN
1   2021-01-02  14.6  10.0  18.8   NaN   NaN  111.0   8.0   NaN  1018.8   NaN
2   2021-01-03  14.7  11.0  19.3   NaN   NaN  116.0   6.9   NaN  1019.2   NaN
3   2021-01-04  15.4  12.5  18.8   NaN   NaN  146.0  12.1   NaN  1014.8   NaN
4   2021-01-05  13.4   8.6  17.6   NaN   NaN  316.0   9.6   NaN  1017.0   NaN
..         ...   ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
360 2021-12-27  13.0   8.9  17.5   NaN   NaN  152.0   8.3   NaN  1019.3   NaN
361 2021-12-28  13.6  10.8  17.0   NaN   NaN  165.0  11.6   NaN  1013.6   NaN
362 2021-12-29  13.7  11.7  16.4   NaN   NaN  158.0   8.9   NaN  1007.2   NaN
363 2021-12-30  12.6  11.1  15.1   NaN   NaN   17.0   8.8   NaN  1003.5   NaN
364 2021-12-31  14.3  11.7  16.4   NaN   NaN   13.0  15.2   NaN  1008.9   NaN

[365 rows x 11 columns]


### Not all coordinates have data in meteostat

#### Using nearby stations

In [21]:
### The Point Class: latitude, longitude, altitude
#https://dev.meteostat.net/python/point.html

from meteostat import Point, Daily
from datetime import datetime

import plotly.express as px


# Set time period
start = datetime(2022, 1, 1)
end = datetime(2022, 12, 31)

# Create Point (near Istambul in Turkey)
location = Point(40 , 30)

# Get daily data
data = Daily(location, start, end)
data = data.fetch()

print(data) #no data is queried
print(type(data))

Empty DataFrame
Columns: [tavg, tmin, tmax, prcp, snow, wdir, wspd, wpgt, pres, tsun]
Index: []
<class 'pandas.core.frame.DataFrame'>


In [24]:
from meteostat import Stations

# Get nearby weather stations
stations = Stations()
stations = stations.nearby(40 , 30)
station = stations.fetch(1) 

nearest_lat = station['latitude'].values[0]
nearest_lon = station['longitude'].values[0]

In [28]:
# Create Point of nearby location for a station
location = Point(nearest_lat , nearest_lon) # ---> 42.25 and 29.55 is the closest to our (40,30) input.

# Get daily data
data = Daily(location, start, end)
data = data.fetch()

print(data) #no data is queried
print(type(data))
print(nearest_lat,nearest_lon)

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2022-01-01   7.4   6.0  10.0   2.6   NaN   10.0   3.0   NaN  1019.4   NaN
2022-01-02   7.8   4.0  11.0   2.1   NaN  357.0   4.4   NaN  1023.0   NaN
2022-01-03   3.5   0.0   9.0   0.0   NaN  169.0   3.8   NaN  1019.7   NaN
2022-01-04   2.3   0.0   8.0   0.0   NaN  252.0   2.8   NaN  1016.8   NaN
2022-01-05   4.7  -2.0  17.0   0.0   NaN  226.0   3.2   NaN  1014.1   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2022-12-27   2.3  -3.0  13.0   0.0   NaN  322.0   1.8   NaN  1026.4   NaN
2022-12-28   7.0   3.1  13.0   1.4   NaN   28.0   4.4   NaN  1027.1   NaN
2022-12-29   6.0   0.0  13.8   0.0   NaN  251.0   2.9   NaN  1026.8   NaN
2022-12-30   3.6  -1.0  13.0   0.0   NaN   17.0   2.4   NaN  1027.7   NaN
2022-12-31   4.5   0.0  11.0   0.0   NaN   83.0   2.1   NaN  1030.3   NaN

[365 rows x 10 columns]
<class 'panda

#### Downloading csv with valid coordinates

In [1]:
from meteostat import Point, Daily
from datetime import datetime
import pandas as pd

def find_valid_coordinates(lat_range, lon_range, start, end):
    valid_coordinates = []

    for lat in lat_range:
        for lon in lon_range:
            location = Point(lat, lon)
            data = Daily(location, start, end)
            data = data.fetch()

            if not data.empty:
                valid_coordinates.append((lat, lon))

    return valid_coordinates

# Set the range of Polish latitude and longitude
polish_lat_range = [49, 50, 51, 52, 53, 54]
polish_lon_range = [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

# Set time period
start = datetime(2023, 1, 1)
end = datetime(2023, 1, 31)

# Find valid coordinates
valid_coords = find_valid_coordinates(polish_lat_range, polish_lon_range, start, end)
print(valid_coords)


[(49, 14), (49, 15), (49, 16), (49, 17), (49, 20), (49, 21), (49, 22), (49, 24), (50, 14), (50, 15), (50, 16), (50, 18), (50, 19), (50, 20), (50, 21), (50, 22), (50, 23), (50, 24), (51, 14), (51, 15), (51, 16), (51, 17), (51, 19), (51, 21), (51, 24), (52, 14), (52, 15), (52, 16), (52, 18), (52, 19), (52, 21), (52, 22), (52, 24), (53, 14), (53, 15), (53, 17), (53, 19), (53, 20), (53, 22), (53, 23), (54, 14), (54, 16), (54, 19), (54, 21), (54, 23)]


In [4]:
len(valid_coords)

45

In [7]:
type(valid_coords)

list

In [8]:
# writing the existing locations in csv

import csv

with open('valid_coords.csv', 'w', newline='') as csvfile:
    coord_writer = csv.writer(csvfile)
    coord_writer.writerow(['lat', 'lon'])
    for coord in valid_coords:
        coord_writer.writerow(coord)

In [9]:
import pandas as pd

# Load valid coordinates from the CSV file
valid_coords = pd.read_csv('valid_coords.csv')

## DASH

### V1

#### V1.0

In [26]:
# import dash
# from dash import html #import dash_core_components as dcc
# from dash import dcc
# from dash.dependencies import Input, Output
# from datetime import datetime, timedelta, date
# from meteostat import Point, Daily

# import plotly.express as px
# import dash_leaflet as dl
# from jupyter_dash import JupyterDash

app = JupyterDash(__name__, external_stylesheets=['https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'])

def plot_tmax_boxplot(lat, lon, start, end, variable):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data=data.reset_index()

    data['Month'] = data['time'].dt.month

    fig = px.box(data, x='Month', y=variable, title='Monthly Tmax Boxplot')
    return fig

def plot_weather_data(lat, lon, start, end):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data=data.reset_index()

    fig = px.line(data, x='time', y=['tmin','tmax'],
                  title=f'Temperature min/max for the Given Location ({lat:.2f},{lon:.2f})',
                  labels={'time': 'Date', 'tmin': 'Min Temp', 'tmax': 'Max Temp'})

    fig.update_yaxes(title_text='Temperature')


    return (fig)

app.layout = html.Div([
    html.H1('Trip Planner', style={'textAlign': 'center', 'padding': '20px'}),
    dcc.DatePickerRange(
        id='date-picker',
        min_date_allowed=datetime(2000, 1, 1),
        max_date_allowed= date.today() - timedelta(days=7), #datetime(2022, 12, 31),
        start_date=datetime(2021, 1, 1),
        end_date=datetime(2022, 12, 31),
        display_format='MMM DD, YYYY'
    ),
    dl.Map(
        [dl.TileLayer(), dl.LayerGroup(id="layer")],
        id='map',
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"},
        center=[35, 25],
        zoom=4,
        #click_lat_lng=True,
    ),
    dcc.Graph(id='weather-plot'),
    html.Label('Select variable for boxplot:'),
    dcc.Dropdown(
        id='boxplot-variable',
        options=[
            {'label': 'Tmax', 'value': 'tmax'},
            {'label': 'Tmin', 'value': 'tmin'}
        ],
        value='tmax'
    ),
    dcc.Graph(id='tmax-boxplot')
])

@app.callback(
    Output('layer', 'children'),
    [Input('map', 'click_lat_lng')],
)
def update_markers(click_lat_lng):
    if not click_lat_lng:
        click_lat_lng = [35, 25]
    return [dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})"))]

@app.callback(
    [Output('weather-plot', 'figure'), Output('tmax-boxplot', 'figure')],
    [Input('map', 'click_lat_lng'), Input('date-picker', 'start_date'), Input('date-picker', 'end_date'), Input('boxplot-variable', 'value')],
)
def update_weather_plots(click_lat_lng, start_date, end_date, boxplot_variable):
    if not click_lat_lng:
        lat, lon = 35, 25
    else:
        lat = int(click_lat_lng[0])
        lon = int(click_lat_lng[1])
    
   
    start = datetime(int(start_date[0:4]),
                    int(start_date[5:7]),
                    int(start_date[8:10]))
    end = datetime(2022, 12, 31)
    
    line_plot = plot_weather_data(lat, lon, start, end)
    
    
    box_plot = plot_tmax_boxplot(lat, lon, start, end, boxplot_variable)
    
    return line_plot, box_plot
    

# Start of the application
if __name__ == '__main__':
    app.run_server(mode="inline", port=8054)


#### V1.1: Temp, Wind, Rain

In [5]:
import dash
from dash import html 
from dash import dcc
from dash.dependencies import Input, Output
from datetime import datetime, timedelta, date
from meteostat import Point, Daily

import plotly.express as px
import dash_leaflet as dl
from jupyter_dash import JupyterDash

app = JupyterDash(__name__, external_stylesheets=['https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'])

def plot_tmax_boxplot(lat, lon, start, end, variable):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    data['month'] = data['time'].dt.month_name()

    fig = px.box(data, x='month', y=variable, title='Monthly Summaries -  Boxplot')

    fig.update_xaxes(title_text='Month')
    fig.update_yaxes(title_text=('Max Temperature (°C)' if variable == 'tmax' else
                              'Min Temperature (°C)' if variable == 'tmin' else
                              'Wind (km/h)' if variable == 'wspd' else
                              'Rain (mm)'))
    return fig

def plot_weather_data(lat, lon, start, end):
    location = Point(lat, lon, 70)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    # Rename the columns
    data = data.rename(columns={'tmin': 'Tmin',
                           'tmax': 'Tmax'})

    fig = px.line(data, x='time', y=['Tmin', 'Tmax'],
                  title=f'Temperature Min/Max for the Given Location ({lat:.2f}, {lon:.2f})',
                  labels={'time': 'Date'})
    fig.update_yaxes(title_text='Temperature °C')

    fig.data[0].name = 'Tmin'
    fig.data[1].name = 'Tmax'

    # Update the legend title
    fig.update_layout(legend_title='Legend')

    return fig


app.layout = html.Div([
    html.H1('Trip Planner', style={'textAlign': 'center', 'padding': '20px'}),
    dcc.DatePickerRange(
        id='date-picker',
        min_date_allowed=datetime(2000, 1, 1),
        max_date_allowed= date.today() - timedelta(days=7),
        start_date=datetime(2021, 1, 1),
        end_date=datetime(2022, 12, 31),
        display_format='MMM DD, YYYY',
        style = {
                        'font-size': '6px','display': 'inline-block', 'border-radius' : '2px', 
                        'border' : '1px solid #ccc', 'color': '#333', 
                        'border-spacing' : '0', 'border-collapse' :'separate'
                        } 
    ),
    dl.Map(
        [dl.TileLayer(), dl.LayerGroup(id="layer")],
        id='map',
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"},
        center=[35, 25],
        zoom=4,
        #click_lat_lng=True,
    ),
    dcc.Graph(id='weather-plot'),
    html.Label('Select a variable to display in the Boxplot:'),
    dcc.Dropdown(
        id='boxplot-variable',
        options=[
            {'label': 'Temperature Max °C', 'value': 'tmax'},
            {'label': 'Temperature Min °C', 'value': 'tmin'},
            {'label': 'Wind (km/h)', 'value': 'wspd'},
            {'label': 'Rain (mm)', 'value': 'prcp'},
        ],
        value='tmax'
    ),
    dcc.Graph(id='tmax-boxplot'),
    html.Div([
        html.H4("About"),
        html.A("My Blog - FossEngineer", href="https://fossengineer.com", target="_blank"),
        html.Br(),
        html.A("About this App - FossEngineer", href="https://fossengineer.com/python-trip-planner/", target="_blank"),
        html.Br(),
        html.A("Source Code", href="https://github.com/JAlcocerT/Py_Trip_Planner/", target="_blank"),
    ], style={'float': 'right'})
])

@app.callback(
    Output('layer', 'children'),
    [Input('map', 'click_lat_lng')],
)
def update_markers(click_lat_lng):
    if not click_lat_lng:
        click_lat_lng = [35, 25]
    return [dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})"))]

@app.callback(
    [Output('weather-plot', 'figure'), Output('tmax-boxplot', 'figure')],
    [Input('map', 'click_lat_lng'), Input('date-picker', 'start_date'), Input('date-picker', 'end_date'), Input('boxplot-variable', 'value')],
)
def update_weather_plots(click_lat_lng, start_date, end_date, boxplot_variable):
    if not click_lat_lng:
        lat, lon = 35, 25
    else:
        lat = int(click_lat_lng[0])
        lon = int(click_lat_lng[1])
    
   
    start = datetime(int(start_date[0:4]),
                    int(start_date[5:7]),
                    int(start_date[8:10]))
    end = datetime(2022, 12, 31)
    
    line_plot = plot_weather_data(lat, lon, start, end)
    
    
    box_plot = plot_tmax_boxplot(lat, lon, start, end, boxplot_variable)
    
    return line_plot, box_plot
    

# Start of the application
if __name__ == '__main__':
    app.run_server(mode="inline", port=8054)

#### V1.2 Adding nearby locations

In [43]:
import dash
from dash import html 
from dash import dcc
from dash.dependencies import Input, Output

from datetime import datetime, timedelta, date

from meteostat import Point, Daily
from meteostat import Stations #new in v1.2

import plotly.express as px
import dash_leaflet as dl

from jupyter_dash import JupyterDash

app = JupyterDash(__name__, external_stylesheets=['https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'])

def plot_tmax_boxplot(lat, lon, start, end, variable):
    location = Point(lat, lon)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    data['month'] = data['time'].dt.month_name()

    fig = px.box(data, x='month', y=variable, title='Monthly Summaries -  Boxplot')

    fig.update_xaxes(title_text='Month')
    fig.update_yaxes(title_text=('Max Temperature (°C)' if variable == 'tmax' else
                              'Min Temperature (°C)' if variable == 'tmin' else
                              'Wind (km/h)' if variable == 'wspd' else
                              'Rain (mm)'))
    return fig

def plot_weather_data(lat, lon, start, end):
    location = Point(lat, lon)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    # Rename the columns
    data = data.rename(columns={'tmin': 'Tmin',
                           'tmax': 'Tmax'})

    fig = px.line(data, x='time', y=['Tmin', 'Tmax'],
                  title=f'Temperature Min/Max for the Given Location ({lat:.2f}, {lon:.2f})',
                  labels={'time': 'Date'})
    fig.update_yaxes(title_text='Temperature °C')

    fig.data[0].name = 'Tmin'
    fig.data[1].name = 'Tmax'

    # Update the legend title
    fig.update_layout(legend_title='Legend')

    return fig


app.layout = html.Div([
    html.H1('Trip Planner', style={'textAlign': 'center', 'padding': '20px'}),
    dcc.DatePickerRange(
        id='date-picker',
        min_date_allowed=datetime(2000, 1, 1),
        max_date_allowed= date.today() - timedelta(days=7),
        start_date=datetime(2021, 1, 1),
        end_date=datetime(2022, 12, 31),
        display_format='MMM DD, YYYY',
        style = {
                        'font-size': '6px','display': 'inline-block', 'border-radius' : '2px', 
                        'border' : '1px solid #ccc', 'color': '#333', 
                        'border-spacing' : '0', 'border-collapse' :'separate'
                        } 
    ),
    dl.Map(
        [dl.TileLayer(), dl.LayerGroup(id="layer")],
        id='map',
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"},
        center=[35, 25],
        zoom=4,
        #click_lat_lng=True,
    ),
    dcc.Graph(id='weather-plot'),
    html.Label('Select a variable to display in the Boxplot:'),
    dcc.Dropdown(
        id='boxplot-variable',
        options=[
            {'label': 'Temperature Max °C', 'value': 'tmax'},
            {'label': 'Temperature Min °C', 'value': 'tmin'},
            {'label': 'Wind (km/h)', 'value': 'wspd'},
            {'label': 'Rain (mm)', 'value': 'prcp'},
        ],
        value='tmax'
    ),
    dcc.Graph(id='tmax-boxplot'),
    html.Div([
        html.H4("About"),
        html.A("My Blog - FossEngineer", href="https://fossengineer.com", target="_blank"),
        html.Br(),
        html.A("About this App - FossEngineer", href="https://fossengineer.com/python-trip-planner/", target="_blank"),
        html.Br(),
        html.A("Source Code", href="https://github.com/JAlcocerT/Py_Trip_Planner/", target="_blank"),
    ], style={'float': 'right'})
])

@app.callback(
    Output('layer', 'children'),
    [Input('map', 'click_lat_lng')],
)
# def update_markers(click_lat_lng):
#     if not click_lat_lng:
#         click_lat_lng = [35, 25]
#     return [dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})"))]

def update_markers(click_lat_lng):
    if not click_lat_lng:
        click_lat_lng = [35, 25]
    return [
        dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})")),
        #dl.Marker(position=[nearest_alt, nearest_lon], children=dl.Tooltip(f"({nearest_alt:.2f}, {nearest_lon:.2f})"))
        dl.CircleMarker(center=[nearest_alt, nearest_lon], color="#188399",)
    ]

@app.callback(
    [Output('weather-plot', 'figure'), Output('tmax-boxplot', 'figure')],
    [Input('map', 'click_lat_lng'), Input('date-picker', 'start_date'), Input('date-picker', 'end_date'), Input('boxplot-variable', 'value')],
)
def update_weather_plots(click_lat_lng, start_date, end_date, boxplot_variable):
    global nearest_alt, nearest_lon
    if not click_lat_lng:
        lat, lon = 35, 25
    else:
        lat = int(click_lat_lng[0])
        lon = int(click_lat_lng[1])
    
   # Get nearby weather stations  - V1.2
    stations = Stations()
    stations = stations.nearby(lat, lon) 
    station = stations.fetch(1) #the closest station to the input location

    nearest_alt = station['latitude'].values[0]
    nearest_lon = station['longitude'].values[0]
    ###

    start = datetime(int(start_date[0:4]),
                    int(start_date[5:7]),
                    int(start_date[8:10]))
    end = datetime(2022, 12, 31)
    
    #line_plot = plot_weather_data(lat, lon, start, end)
    line_plot = plot_weather_data(nearest_alt, nearest_lon, start, end)
    
    #box_plot = plot_tmax_boxplot(lat, lon, start, end, boxplot_variable)
    box_plot = plot_tmax_boxplot(nearest_alt, nearest_lon, start, end, boxplot_variable)
    
    return line_plot, box_plot
    

# Start of the application
if __name__ == '__main__':
    app.run_server(mode="inline", port=8054)

### V2 Adding Weather Forecast with OpenMeteo

In [ ]:
import dash
from dash import html 
from dash import dcc
from dash.dependencies import Input, Output

from datetime import datetime, timedelta, date

from meteostat import Point, Daily
from meteostat import Stations #new in v1.2

import plotly.express as px
import dash_leaflet as dl

from jupyter_dash import JupyterDash

app = JupyterDash(__name__, external_stylesheets=['https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'])

def plot_tmax_boxplot(lat, lon, start, end, variable):
    location = Point(lat, lon)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    data['month'] = data['time'].dt.month_name()

    fig = px.box(data, x='month', y=variable, title='Monthly Summaries -  Boxplot')

    fig.update_xaxes(title_text='Month')
    fig.update_yaxes(title_text=('Max Temperature (°C)' if variable == 'tmax' else
                              'Min Temperature (°C)' if variable == 'tmin' else
                              'Wind (km/h)' if variable == 'wspd' else
                              'Rain (mm)'))
    return fig

def plot_weather_data(lat, lon, start, end):
    location = Point(lat, lon)
    
    data = Daily(location, start, end)
    data = data.fetch()

    data = data.reset_index()

    # Rename the columns
    data = data.rename(columns={'tmin': 'Tmin',
                           'tmax': 'Tmax'})

    fig = px.line(data, x='time', y=['Tmin', 'Tmax'],
                  title=f'Temperature Min/Max for the Given Location ({lat:.2f}, {lon:.2f})',
                  labels={'time': 'Date'})
    fig.update_yaxes(title_text='Temperature °C')

    fig.data[0].name = 'Tmin'
    fig.data[1].name = 'Tmax'

    # Update the legend title
    fig.update_layout(legend_title='Legend')

    return fig


app.layout = html.Div([
    html.H1('Trip Planner', style={'textAlign': 'center', 'padding': '20px'}),
    dcc.DatePickerRange(
        id='date-picker',
        min_date_allowed=datetime(2000, 1, 1),
        max_date_allowed= date.today() - timedelta(days=7),
        start_date=datetime(2021, 1, 1),
        end_date=datetime(2022, 12, 31),
        display_format='MMM DD, YYYY',
        style = {
                        'font-size': '6px','display': 'inline-block', 'border-radius' : '2px', 
                        'border' : '1px solid #ccc', 'color': '#333', 
                        'border-spacing' : '0', 'border-collapse' :'separate'
                        } 
    ),
    dl.Map(
        [dl.TileLayer(), dl.LayerGroup(id="layer")],
        id='map',
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"},
        center=[35, 25],
        zoom=4,
        #click_lat_lng=True,
    ),
    dcc.Graph(id='weather-plot'),
    html.Label('Select a variable to display in the Boxplot:'),
    dcc.Dropdown(
        id='boxplot-variable',
        options=[
            {'label': 'Temperature Max °C', 'value': 'tmax'},
            {'label': 'Temperature Min °C', 'value': 'tmin'},
            {'label': 'Wind (km/h)', 'value': 'wspd'},
            {'label': 'Rain (mm)', 'value': 'prcp'},
        ],
        value='tmax'
    ),
    dcc.Graph(id='tmax-boxplot'),
    html.Div([
        html.H4("About"),
        html.A("My Blog - FossEngineer", href="https://fossengineer.com", target="_blank"),
        html.Br(),
        html.A("About this App - FossEngineer", href="https://fossengineer.com/python-trip-planner/", target="_blank"),
        html.Br(),
        html.A("Source Code", href="https://github.com/JAlcocerT/Py_Trip_Planner/", target="_blank"),
    ], style={'float': 'right'})
])

@app.callback(
    Output('layer', 'children'),
    [Input('map', 'click_lat_lng')],
)
# def update_markers(click_lat_lng):
#     if not click_lat_lng:
#         click_lat_lng = [35, 25]
#     return [dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})"))]

def update_markers(click_lat_lng):
    if not click_lat_lng:
        click_lat_lng = [35, 25]
    return [
        dl.Marker(position=click_lat_lng, children=dl.Tooltip(f"({click_lat_lng[0]:.2f}, {click_lat_lng[1]:.2f})")),
        #dl.Marker(position=[nearest_alt, nearest_lon], children=dl.Tooltip(f"({nearest_alt:.2f}, {nearest_lon:.2f})"))
        dl.CircleMarker(center=[nearest_alt, nearest_lon], color="#188399",)
    ]

@app.callback(
    [Output('weather-plot', 'figure'), Output('tmax-boxplot', 'figure')],
    [Input('map', 'click_lat_lng'), Input('date-picker', 'start_date'), Input('date-picker', 'end_date'), Input('boxplot-variable', 'value')],
)
def update_weather_plots(click_lat_lng, start_date, end_date, boxplot_variable):
    global nearest_alt, nearest_lon
    if not click_lat_lng:
        lat, lon = 35, 25
    else:
        lat = int(click_lat_lng[0])
        lon = int(click_lat_lng[1])
    
   # Get nearby weather stations  - V1.2
    stations = Stations()
    stations = stations.nearby(lat, lon) 
    station = stations.fetch(1) #the closest station to the input location

    nearest_alt = station['latitude'].values[0]
    nearest_lon = station['longitude'].values[0]
    ###

    start = datetime(int(start_date[0:4]),
                    int(start_date[5:7]),
                    int(start_date[8:10]))
    end = datetime(2022, 12, 31)
    
    #line_plot = plot_weather_data(lat, lon, start, end)
    line_plot = plot_weather_data(nearest_alt, nearest_lon, start, end)
    
    #box_plot = plot_tmax_boxplot(lat, lon, start, end, boxplot_variable)
    box_plot = plot_tmax_boxplot(nearest_alt, nearest_lon, start, end, boxplot_variable)
    
    return line_plot, box_plot
    

# Start of the application
if __name__ == '__main__':
    app.run_server(mode="inline", port=8054)